In [7]:
import numpy as np # linear algebra
import pandas as pd
import joblib
df = pd.read_csv('Car_Prices_Poland_Kaggle.csv/Car_Prices_Poland_Kaggle.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Car_Prices_Poland_Kaggle.csv/Car_Prices_Poland_Kaggle.csv'

In [ ]:
df1= df.drop('Unnamed: 0' , axis=1)
df1=df1.drop(['generation_name', 'province', 'city'],axis=1)
df1

,mark,model,year,mileage,vol_engine,fuel,price
0,opel,combo,2015,139568,1248,Diesel,35900
1,opel,combo,2018,31991,1499,Diesel,78501
2,opel,combo,2015,278437,1598,Diesel,27000
3,opel,combo,2016,47600,1248,Diesel,30800
4,opel,combo,2014,103000,1400,CNG,35900
...,...,...,...,...,...,...,...
117922,volvo,xc-90,2020,40000,1969,Hybrid,222790
117923,volvo,xc-90,2017,51000,1969,Diesel,229900
117924,volvo,xc-90,2016,83500,1969,Gasoline,135000
117925,volvo,xc-90,2017,174000,1969,Diesel,154500


In [ ]:
df1.shape

(117927, 7)

In [ ]:
# get the number of missing data points per column
missing_values = df1.isnull().sum()
# look at the # of missing points
missing_values

mark          0
model         0
year          0
mileage       0
vol_engine    0
fuel          0
price         0
dtype: int64

In [ ]:
# # Handling missing values in 'generation_name'
# df1['generation_name'].fillna('Unknown', inplace=True)

# # Removing duplicates
df1.drop_duplicates(inplace=True)

In [ ]:
df1.isnull().sum()

mark          0
model         0
year          0
mileage       0
vol_engine    0
fuel          0
price         0
dtype: int64

In [ ]:
df1.shape

(110023, 7)

In [ ]:
# Resetting index
df1.reset_index(drop=True, inplace=True)
df1

,mark,model,year,mileage,vol_engine,fuel,price
0,opel,combo,2015,139568,1248,Diesel,35900
1,opel,combo,2018,31991,1499,Diesel,78501
2,opel,combo,2015,278437,1598,Diesel,27000
3,opel,combo,2016,47600,1248,Diesel,30800
4,opel,combo,2014,103000,1400,CNG,35900
...,...,...,...,...,...,...,...
110018,volvo,xc-90,2017,51000,1969,Diesel,229900
110019,volvo,xc-90,2016,83500,1969,Gasoline,135000
110020,volvo,xc-90,2017,174000,1969,Diesel,154500
110021,volvo,xc-90,2016,189020,1969,Gasoline,130000


In [ ]:
# Converting data types
df1['year'] = df1['year'].astype(int)
df1['mileage'] = df1['mileage'].astype(int)
df1['vol_engine'] = df1['vol_engine'].astype(int)
df1['price'] = df1['price'].astype(float)


In [ ]:
df1['year']

0         2015
1         2018
2         2015
3         2016
4         2014
          ... 
110018    2017
110019    2016
110020    2017
110021    2016
110022    2016
Name: year, Length: 110023, dtype: int32

In [ ]:
# Define a function to calculate IQR and detect outliers
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    outliers = df[(df[column] > upper_bound) | (df[column] < lower_bound)]
    return outliers

In [ ]:
# Detect outliers in 'mileage'
mileage_outliers = detect_outliers(df1, 'mileage')
print(f"Number of outliers in 'mileage': {len(mileage_outliers)}")

Number of outliers in 'mileage': 468


In [ ]:
# Detect outliers in 'vol_engine'
vol_engine_outliers = detect_outliers(df1, 'vol_engine')
print(f"Number of outliers in 'vol_engine': {len(vol_engine_outliers)}")

Number of outliers in 'vol_engine': 9887


In [ ]:
# Remove outliers from the dataset
df1 = df1[~df1.index.isin(mileage_outliers.index)]
df1 = df1[~df1.index.isin(vol_engine_outliers.index)]

In [ ]:
df1.shape



(99777, 7)

In [ ]:
# Scaling for algorithms that is sensitive to the magnitude of variables(KNN,SVM,or neural networks)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df1[['mileage', 'vol_engine']])  # Apply to numerical columns
df1[['mileage', 'vol_engine']] = df_scaled
joblib.dump(scaler, "cars_stdscaler.joblib")



['cars_stdscaler.joblib']

In [ ]:
df_scaled

array([[-0.02158922, -1.27756604],
       [-1.27058024, -0.54681627],
       [ 1.59070833, -0.25859226],
       ...,
       [ 0.37817335,  0.82151994],
       [ 0.55255863,  0.82151994],
       [-0.53903284,  0.82151994]])

In [ ]:
# Normalization algorithms that are sensitive to the distribution of values and require uniformity like gradient descent
from sklearn.preprocessing import  MinMaxScaler
normalizer = MinMaxScaler()
df_normalized = normalizer.fit_transform(df1[['mileage', 'vol_engine']])  # Apply to numerical columns
df1[['mileage', 'vol_engine']] = df_normalized
joblib.dump(normalizer, "cars_minmaxscaler.joblib")

['cars_minmaxscaler.joblib']

In [ ]:
df_normalized

array([[0.34675117, 0.2045221 ],
       [0.07947846, 0.33350462],
       [0.69176818, 0.38437821],
       ...,
       [0.43229673, 0.57502569],
       [0.46961359, 0.57502569],
       [0.23602295, 0.57502569]])

In [ ]:
# Apply PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
df_reduced = pca.fit_transform(df_scaled)

In [ ]:
df_reduced

array([[-0.9186415 , -0.88810972],
       [-1.2850934 ,  0.51177841],
       [ 0.9419483 , -1.30765299],
       ...,
       [ 0.84831126,  0.31349338],
       [ 0.97162028,  0.19018437],
       [ 0.19974855,  0.9620561 ]])

In [ ]:
df_reduced.shape

(99777, 2)

In [ ]:
# Encoding categorical variables using Label Encoding for high cardinality features
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
high_cardinality_features = ['mark', 'model']
for feature in high_cardinality_features:
    df1[feature] = label_encoder.fit_transform(df1[feature])
    
joblib.dump(label_encoder, "cars_label_encoder.joblib")    

['cars_label_encoder.joblib']

In [ ]:
df1

,mark,model,year,mileage,vol_engine,fuel,price
0,15,86,2015,0.346751,0.204522,Diesel,35900.0
1,15,86,2018,0.079478,0.333505,Diesel,78501.0
2,15,86,2015,0.691768,0.384378,Diesel,27000.0
3,15,86,2016,0.118259,0.204522,Diesel,30800.0
4,15,86,2014,0.255899,0.282631,CNG,35900.0
...,...,...,...,...,...,...,...
110018,22,289,2017,0.126706,0.575026,Diesel,229900.0
110019,22,289,2016,0.207451,0.575026,Gasoline,135000.0
110020,22,289,2017,0.432297,0.575026,Diesel,154500.0
110021,22,289,2016,0.469614,0.575026,Gasoline,130000.0


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# # One-Hot Encoding for low cardinality features
df_encoded = pd.get_dummies(df1, columns=[ 'fuel'])
# Save the OneHotEncoder
# Save the column names as a list
encoder = df_encoded.columns.tolist()
joblib.dump(encoder, "one_hot_encoder.joblib")
# one_hot_encoder = OneHotEncoder(sparse=False)

# # Step 2: Fit and transform the DataFrame
# df_encoded = one_hot_encoder.fit_transform(df1[['fuel']])


# # Step 3: Create a new DataFrame from the encoded data
# df_encoded = pd.DataFrame(df_encoded, columns=one_hot_encoder.get_feature_names_out(['fuel']))

# # Step 4: Concatenate the original DataFrame with the encoded DataFrame
# df_encoded = pd.concat([df1.drop(columns=['fuel']), df_encoded], axis=1)

# joblib.dump(label_encoder, "cars_onehot_encoder.joblib") 


['one_hot_encoder.joblib']

In [ ]:
df_encoded

,mark,model,year,mileage,vol_engine,price,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_Gasoline,fuel_Hybrid,fuel_LPG
0,15,86,2015,0.346751,0.204522,35900.0,0,1,0,0,0,0
1,15,86,2018,0.079478,0.333505,78501.0,0,1,0,0,0,0
2,15,86,2015,0.691768,0.384378,27000.0,0,1,0,0,0,0
3,15,86,2016,0.118259,0.204522,30800.0,0,1,0,0,0,0
4,15,86,2014,0.255899,0.282631,35900.0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
110018,22,289,2017,0.126706,0.575026,229900.0,0,1,0,0,0,0
110019,22,289,2016,0.207451,0.575026,135000.0,0,0,0,1,0,0
110020,22,289,2017,0.432297,0.575026,154500.0,0,1,0,0,0,0
110021,22,289,2016,0.469614,0.575026,130000.0,0,0,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Splitting the data
X = df_encoded.drop(['price'], axis=1)
y = df_encoded['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)
gb_model.fit(X_train, y_train)

# Predictions
y_pred = gb_model.predict(X_test)

# Model Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
# Save the trained model weights to a file
joblib.dump(gb_model, 'cars_gb_model.joblib')
print("Testing Score:", gb_model.score(X_test, y_test))




Mean Squared Error: 269362180.8729368
R^2 Score: 0.9183128842379736
Testing Score: 0.9183128842379736


In [1]:
import pandas as pd
import joblib



#Using inference on given data
new_data = pd.DataFrame({
    'mark': ['chevrolet'],
    'model': ['cruze'],
    
    'year': [2012],
    'mileage': [220000],
    'vol_engine': [1991],
    'fuel': ['Diesel']
})

normalizer = joblib.load('cars_minmaxscaler.joblib')
new_data_normalized = normalizer.fit_transform(new_data[['mileage', 'vol_engine']]) # Apply to numerical columns
new_data[['mileage', 'vol_engine']] = new_data_normalized


scaler = joblib.load('cars_stdscaler.joblib')
new_data_scaled = scaler.fit_transform(new_data[['mileage', 'vol_engine']]) # Apply to numerical columns
new_data[['mileage', 'vol_engine']] = new_data_scaled

#Preprocess
label_encoder = joblib.load('cars_label_encoder.joblib')
high_cardinality_features = ['mark', 'model']
for feature in high_cardinality_features:
    new_data[feature] = label_encoder.fit_transform(new_data[feature])

# Load the saved OneHotEncoder
encoder = joblib.load("one_hot_encoder.joblib")
# One-Hot Encoding for low cardinality features
df_encoded = pd.get_dummies(new_data, columns=['fuel'])

# Reorder the columns based on the saved encoder
df_encoded = df_encoded.reindex(columns=encoder)

df_encoded=df_encoded.drop('price',axis=1)
# Print the encoded DataFrame
print(df_encoded)


model=joblib.load('cars_gb_model.joblib')

# Replace NaN values with 0
df_encoded = df_encoded.fillna(0)


#The predicition
predictions=model.predict(df_encoded)

print(predictions)

   mark  model  year  mileage  vol_engine  fuel_CNG  fuel_Diesel  \
0     0      0  2012      0.0         0.0       NaN         True   

   fuel_Electric  fuel_Gasoline  fuel_Hybrid  fuel_LPG  
0            NaN            NaN          NaN       NaN  
[39430.44932405]
